# 00 - Kanton Zürich - Map Widget
### Author: Daniel Herrera-Russert
#### February 03, 2025

## 1. Querying Geothermal Probe Allowance Based on Location

To enable a seamless querying mechanism for determining the allowance status of a given location, we introduce a function that performs spatial queries on the processed geographical allowability dataset. The ultimate goal is to integrate this functionality into a UI-based application featuring an interactive map widget. 

Since standard map tile providers operate in **WGS84 coordinates**, while Swiss geospatial datasets maintain higher accuracy in **LV95 (Swiss Coordinate System)**, we will preserve all datasets in LV95 for compatibility with other Swiss geodata. However, user inputs will be provided in WGS84 and must be converted accordingly before performing spatial operations.

The following code is based on a function which takes **WGS84 coordinates as input**, converts them to LV95, and checks their intersection with the preprocessed restriction polygons. If a match is found, it returns the corresponding restriction category as a string. 

Additionaly, the depth limitation function, which is imported as a package named `depth_query`, retrieves elevation and maximum allowable borehole depth for a given coordinate in the Canton of Zürich by making an HTTP GET request to the Zürich maps API. First, the input coordinates in WGS84 format are transformed into the Swiss LV95 coordinate system using `pyproj.Transformer`, as the API requires queries in LV95. The function then constructs a URL with the converted coordinates and sends a request using the `requests` library. The response, which is in HTML format, is parsed using `BeautifulSoup` to extract relevant information. Specifically, it searches for numerical values associated with elevation (marked as "Höhe") and depth constraints (marked as "Meter ab Terrain") using regular expressions.

In [1]:
# !pip install jupyterlab-widgets ipywidgets

In [2]:
import ipyleaflet
import geopandas as gpd
import json
import pyproj
from shapely.geometry import Point
import ipywidgets as widgets
from scripts.depth_query import get_depth_info # Depth calculator

In [3]:
# Load the cleaned dataset
zh_cleaned_gdf = gpd.read_file("data/transformed/zh_combined_restrictions.geojson")

# Ensure the dataset is in LV95 (Swiss Coordinate System)
zh_cleaned_gdf = zh_cleaned_gdf.to_crs("EPSG:2056")

# Load the Zürich boundary GeoJSON
boundary_gdf = gpd.read_file("data/raw/zh_boundary.geojson")
boundary_geojson = json.loads(boundary_gdf.to_json())  # Convert to JSON format

In [4]:
# Initialize ipyleaflet map centered on Zürich
m = ipyleaflet.Map(center=(47.3769, 8.5417), zoom=10)

# Create a GeoJSON layer for the boundary
boundary_layer = ipyleaflet.GeoJSON(
    data=boundary_geojson,
    style={"color": "gray", "fillOpacity": 0.2},  # Boundary style
    name="Zürich Boundary"
)

# Add the boundary layer to the map
m.add_layer(boundary_layer)

# Create an output widget
output = widgets.Output()

# Create a draggable marker
marker = ipyleaflet.Marker(location=(47.3769, 8.5417), draggable=True)
m.add_layer(marker)

# Define function for coordinate conversion & restriction query
def check_restriction(lat, lon):
    """Takes WGS84 latitude & longitude and returns the restriction status."""
    
    # Convert user coordinates from WGS84 to LV95
    wgs84_to_lv95 = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:2056", always_xy=True)
    lv95_x, lv95_y = wgs84_to_lv95.transform(lon, lat)
    
    # Create a point in LV95
    user_point_lv95 = Point(lv95_x, lv95_y)

    # Check if the point falls within any restriction area
    matching_row = zh_cleaned_gdf[zh_cleaned_gdf.contains(user_point_lv95)]

    if not matching_row.empty:
        return matching_row.iloc[0]["restrictions"]
    else:
        return "❌ Outside of Canton Limits!"

# Function to handle marker drag event
def on_marker_drag(change):
    """Triggered when marker is dragged to a new location."""
    lat, lon = marker.location  # Get new marker location
    
    # Query restriction status
    restriction_value = check_restriction(lat, lon)

    # Query depth information
    elevation, depth_max = get_depth_info(lat, lon)
    
    # Display result
    with output:
        output.clear_output()
        print(f"📍 Dragged to: ({lat:.5f}, {lon:.5f}) → Status: {restriction_value}")
        print(f"🏔️ Elevation: {elevation} | 🔽 Max Depth: {depth_max}")

# Attach event listener for dragging
marker.observe(on_marker_drag, "location")

# Display map and output
display(m, output)

Map(center=[47.3769, 8.5417], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

Output()